# Get water speciation from T, pH and DIC 
# inverse calculations
Carbonate system calculation.Input T, pH, DIC. Assuming metal cations + carbonate alkalinity.

With this script you can calculate the charge contribution or contribution to alkalinity of carbonate species for a given pH.

In [270]:
from phreeqpython import PhreeqPython

import numpy as np


from matplotlib import pyplot as plt

import pandas as  pd

In [271]:
# use the vitens database for all constants


# create new PhreeqPython instance

#pp = PhreeqPython(database='minteq.v4.dat')

pp = PhreeqPython(database='vitens.dat')


In [272]:
# calculate CO2(g) SI (how much CO2 in the gas phase )

#concentration CO2
p=500 # ppm

#partial pressure pCO2 in atm
pCO2=p*1e-6 


 #phreeqc always uses log10 values
input_pCO2=np.log10(pCO2)

print('log10 CO2(g)=',input_pCO2)

log10 CO2(g)= -3.3010299956639813


In [273]:
#include DIC

# david Parkhust answer
# Essentially, carbon input is either Alkalinity or total dissolved inorganic carbon (C(4)).

# C(4) means DIC

# In der Hydrochemie verwendet man oftmals anstelle des DIC die präzisere Bezeichnung C(4) – das ist Kohlenstoff 
# in der Oxidationsstufe IV


DIC=5

#pH=9.53

#phreeqc ignores the charge imbalance created
# it ignores other involved species 
# assumtion about cations to make the result more reliable 

solution2 = pp.add_solution({'units':'mmol/kgw',
                                 'density': 1.000,
                                 'temp': 25.0,
                                 'Ca':DIC/2,
                                 'C(4)':DIC,
                                }).equalize(['CO2(g)'], [input_pCO2])



#make sure  the unit is mol
co2=solution2.total('CO2',units='mol')
hco3=solution2.total('HCO3',units='mol')
co3=solution2.total('CO3',units='mol')

# get the H and OH concentrations
h=solution2.species['H+']   # species are in mol/l
oh=solution2.species['OH-'] 


#get alkalinity
TA_out=solution2

# simple 
dic=co2+hco3+co3


#charge contribution
#carbonate + hydroxide - acid 
TA=hco3+2*co3+oh-h

print(dic)

print('TA=',TA)

#alkalinity
print('pure carbonate alkalinity=',TA)



0.003913594586696361
TA= 0.004169013220244283
pure carbonate alkalinity= 0.004169013220244283


In [274]:
#get the species in the water 
solution2.species

{'CH4': 4.326593857499193e-25,
 'CO2': 1.7256153857395685e-05,
 'CO3-2': 9.101963590581142e-05,
 'Ca+2': 0.0022497777334057646,
 'CaCO3': 0.00017695827336814692,
 'CaHCO3+': 7.313977707935345e-05,
 'CaOH+': 1.2422362455885257e-07,
 'H+': 2.53507610452865e-09,
 'H2': 7.1049269248800196e-15,
 'H2O': 55.50955840300623,
 'HCO3-': 0.0035552207464856532,
 'O2': 0.0,
 'OH-': 4.699413207464101e-06}

In [275]:
#get SI of the phases
solution2.phases['CO2(g)']

-3.3010311978261866

In [276]:
# check whether it is reliable with solution phases



In [277]:
#get SI of the phases
solution2.phases



{'Aragonite': 1.359295222837794,
 'CH4(g)': -21.520262627647465,
 'CO2(g)': -3.3010311978261866,
 'Calcite': 1.5030650702300292,
 'Fix_pH': -8.629212215133954,
 'H2(g)': -11.046715641258274,
 'H2O(g)': -1.5028233204748613,
 'O2(g)': -61.192146977790756,
 'Vaterite': 0.9366335760157298}

# Reconstruction of the result

#Ksp
solubility product

Debye and Hückel developed the following equation to calculate the mean ionic activity coefficient γ±:

$\log\gamma_{\pm}=-\dfrac{1.824\times10^{6}}{(\varepsilon T)^{3/2}}\mid z_{+}z_{-}\mid\sqrt{I} \label{1}$


$\log \gamma_{\pm}=-0.509\mid z_{+}z_{-}\mid\sqrt{I} \label{3}$

Mean molality:

ν is the stoichiometric coefficient of the ions

$m_{\pm}=(m_{+}^{\nu+}m_{-}^{\nu-})^{\frac{1}{\nu}} \nonumber$

In [278]:
# Ksp of Calcite

log_k=-8.336

Ksp=10**log_k

In [279]:
# saturation index

# IAP = Ion activity product
# SP= solubility product
def SI(IAP,SP):
    
    saturation_index=np.log10((IAP/SP))
    
    return saturation_index

def y(I,z_plus,z_minus):
    Y=10**(-0.509*np.abs(z_plus*z_minus)*np.sqrt(I))

    return Y


def mean_m(c1,c2,v1,v2):
    m=((c1**v1)*(c2**v2))**(1/(v1+v2))
    
    return m

In [280]:
# calcualate the IAP

#'CO3-2': 2.5787579856073448e-05,
# 'Ca+2': 0.001195571406233925,

In [281]:
c_CO3=solution2.species['CO3-2']

c_Ca=solution2.species['Ca+2']



# mean ionic molality

# activity coefficient is needed

# Get the ionic strength
ionic_strength = solution2.I

ionic_strength

0.006498157574274969

In [282]:
#calculate the ionic activty product

# IAP =a_Ca2+ * a_CO32- = y*c_Ca2+ * y*c_CO32-

IAP=(y(ionic_strength,2,2)**2)*(c_Ca*c_CO3)



In [283]:
SI(IAP,Ksp)

1.3190261034998305